## Full CSIT : No feedback, No pilot sequence required

In [2]:
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
## System parameters
M = 64 #Number of BS antennas
P = 1 #Power
K =  2 #Number of users
L = 8 #Number of pilots
Lp = 2 #Number of paths
B = 30 #Number of feedback bits per user

## Limited scattering channel parameters
LSF_UE = np.array([0.0,0.0],dtype=np.float32) #Mean of path gains for K users
Mainlobe_UE= np.array([0,0],dtype=np.float32) #Center of the AoD range for K users
HalfBW_UE = np.array([30.0,30.0],dtype=np.float32) #Half of the AoD range for K users

# SNR
snr_dl = 10 #SNR in dB
noise_std_dl = np.float32(np.sqrt(1/2)*np.sqrt(P/10**(snr_dl/10))) #STD of the Gaussian noise (per real dim.)

# Number of Monte-Carlo similations -> batch_size = N_experiment
iter = 10000

### Batch data 생성 함수

In [4]:
## 리스트 인덱싱은 [층, 행, 열], [행, 열]임!
def generate_batch_data(batch_size,M,K,
                        Lp,#number of paths
                        LSF_UE #Mean of path gains for K users
                        ,Mainlobe_UE #Center of the AoD range for K users
                        ,HalfBW_UE #Half of the AoD range for K users
                        ):
    alphaR_input = np.zeros((batch_size,Lp,K))
    alphaI_input = np.zeros((batch_size,Lp,K))
    theta_input = np.zeros((batch_size,Lp,K))
    for kk in range(K): # for the number of users
        alphaR_input[:,:,kk] = np.random.normal(loc=LSF_UE[kk], scale=1.0/np.sqrt(2), size=[batch_size,Lp])
        alphaI_input[:,:,kk] = np.random.normal(loc=LSF_UE[kk], scale=1.0/np.sqrt(2), size=[batch_size,Lp])
        theta_input[:,:,kk] = np.random.uniform(low=Mainlobe_UE[kk]-HalfBW_UE[kk], high=Mainlobe_UE[kk]+HalfBW_UE[kk], size=[batch_size,Lp])
 
    #### Actual Channel
    from0toM = np.float32(np.arange(0, M, 1))
    alpha_act = alphaR_input + 1j*alphaI_input
    theta_act = (np.pi/180)*theta_input
    
    h_act = np.complex128(np.zeros((batch_size,M,K)))
    hR_act = np.float32(np.zeros((batch_size,M,K)))
    hI_act = np.float32(np.zeros((batch_size,M,K)))
    
    for kk in range(K):
        for ll in range(Lp):
            theta_act_expanded_temp = np.tile(np.reshape(theta_act[:,ll,kk],[-1,1]),(1,M))
            response_temp = np.exp(1j*np.pi*np.multiply(np.sin(theta_act_expanded_temp),from0toM))
            alpha_temp = np.reshape(alpha_act[:,ll,kk],[-1,1])
            h_act[:,:,kk] += (1/np.sqrt(Lp))*alpha_temp*response_temp
        hR_act[:,:,kk] = np.real(h_act[:,:,kk])
        hI_act[:,:,kk] = np.imag(h_act[:,:,kk])
        
    h_act = torch.tensor(h_act, dtype = torch.complex64).to(device)
    hR_act = torch.tensor(hR_act).to(device)
    hI_act = torch.tensor(hI_act).to(device)
        
    return(h_act, hR_act, hI_act)

### Precoder, Rate 계산 함수

In [5]:
## Precoder
def ZF_Precoding(h_act):
    H_act = h_act
    V = torch.zeros((H_act.shape[0], H_act.shape[1], H_act.shape[2]), dtype=torch.complex64)
    
    for i in range(H_act.shape[0]):
        V[i, :, :] = torch.linalg.pinv(H_act[i, :, :].T)
        V[i, :, :] = V[i, :, :] / torch.sqrt(torch.trace(V[i, :, :] @ (V[i, :, :].H)))
    
    return V.to(device)
    
def MRT_Precoding(h_act):
    H_act = h_act
    V = torch.zeros((H_act.shape[0], H_act.shape[1], H_act.shape[2]), dtype=torch.complex64)
    
    for i in range(H_act.shape[0]):
        V[i, :, :] = H_act[i, :, :].conj()
        V[i, :, :] = V[i, :, :] / torch.sqrt(torch.trace(V[i, :, :] @ (V[i, :, :].H)))
    
    return V.to(device)

In [6]:
def rate_calc(h_act_user, M, K, k_idx, V, noise_std):
    H_act = h_act_user
    nom_plus_denom = torch.zeros((H_act.shape[0], 1)).to(device) + torch.tensor(2 * noise_std ** 2).to(device)
    
    for kk in range(K):
        product = torch.bmm(H_act.clone().unsqueeze(-1).permute(0, 2, 1), V[:, :, kk].clone().unsqueeze(-1)).squeeze(-1)
        norm2 = torch.abs(product) ** 2
        norm2.to(device)
        
        if kk == k_idx:
            nom = norm2 # Wanted signal power
        nom_plus_denom += norm2
        
    denom = nom_plus_denom - nom
    rate = torch.mean(torch.log2(1 + (nom / denom)))
    
    return rate

### Monte-Carlo Simulation

In [7]:
# Actual channel generation
h_act, _, _ = generate_batch_data(iter, M, K, Lp, LSF_UE, Mainlobe_UE, HalfBW_UE)
rate_MRT, rate_ZF = 0, 0
for kk in range(K):
      rate_MRT += rate_calc(h_act[: ,:, kk], M, K, kk, MRT_Precoding(h_act), noise_std_dl)
      rate_ZF += rate_calc(h_act[: ,:, kk], M, K, kk, ZF_Precoding(h_act), noise_std_dl)
      
# print('MRT rate : '+ rate_MRT + 'bps/Hz, ZF rate : '+ rate_ZF +'bps/Hz')

In [10]:
print(h_act.size())

torch.Size([10000, 64, 2])


In [8]:
print(rate_MRT)

tensor(13.1916, device='cuda:0')


In [9]:
print(rate_ZF)

tensor(15.3562, device='cuda:0')
